# Data Compression Via Dimensionality Reduction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
wine_cols = ['Class label','Alcohol','Malic Acid','Ash','Alcalinity of ash',
             'Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols',
             'Proanthocyanins','Color intensity','Hue',
             'OD280/OD315 of diluted wines','Proline'
            ]
df_wine = pd.read_csv(url,header=None)
df_wine.columns = wine_cols
df_wine.head()

# Linear Dimensionality Reduction

## Unsupervised Dimensionality Reduction via Principal Component Analysis

### Step 1: Create Training and Testing Datasets

In [ ]:
from sklearn.model_selection import train_test_split
X,y = df_wine.iloc[:,1:].values, df_wine.iloc[:,0].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,stratify=y,random_state=0)

### Step 2: Standardize the Features

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

### Step 3: Construct a covariance matrix

In [ ]:
cov_mat = np.cov(X_train_std.T)

### Step 4: Calculate the Eigenpairs for the Covariance Matrix

In [ ]:
eigen_vals,eigen_vecs = np.linalg.eig(cov_mat)
print('\nEigenvalues \n',eigen_vals)

#### Graphing the Variance Explained Ratios

In [ ]:
tot = sum(eigen_vals)
var_exp = [(i/tot) for i in sorted(eigen_vals,reverse=True)]
cum_var_exp = np.cumsum(var_exp)

plt.bar(range(1,14),var_exp,align='center',label='Individual Explained Variance')
plt.step(range(1,14),cum_var_exp,where='mid',label='Cumulative Explained Variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Component Index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

### Step 5: Final Feature Transformations onto the new principal component axes.

In [ ]:
# Sort (eigenvalue,eigenvector) from high to low
eigen_pairs = [(np.abs(eigen_vals[i]),eigen_vecs[:,i]) for i in range(len(eigen_vals))]
eigen_pairs.sort(key=lambda k : k[0],reverse=True)

#Collect the two largest eigenvectors correlating to the two largest eigenvalues
#This creates a 13x2 dim projection matrix
w = np.hstack(
    (
        eigen_pairs[0][1][:,np.newaxis],
        eigen_pairs[1][1][:,np.newaxis]
    )
)
print('Matrix W:\n',w)

In [ ]:
#Transform our training data onto the PCA subspace
X_train_pca = X_train_std.dot(w)

#### Visualizing the newly projected data

In [ ]:
colors = ['r','b','g']
markers = ['o','s','^']

for l,c,m in zip(np.unique(y_train),colors,markers):
    plt.scatter(X_train_pca[y_train==l,0],
                X_train_pca[y_train==l,1],
                c=c,
                label=f'Class {l}',
                marker=m
                )
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

## Unsupervised Dimensionality Reduction via Principal Component Analysis (Implemented using Sklearn)

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X,y,classifier,test_idx=None,resolution=.02):

    #setup marker generator and color map
    markers = ('o','s','^','v','<')
    colors = ('red','blue','lightgreen','gray','cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    #plot decision surface
    x1_min,x1_max = X[:,0].min()-1,X[:,0].max()+1
    x2_min,x2_max = X[:,1].min()-1,X[:,1].max()+1
    xx1,xx2 = np.meshgrid(
        np.arange(x1_min,x1_max,resolution),
        np.arange(x2_min,x2_max,resolution)
    )

    lab = classifier.predict(np.array([xx1.ravel(),xx2.ravel()]).T)
    lab = lab.reshape(xx1.shape)

    plt.contourf(xx1,xx2,lab,alpha=.3,cmap=cmap)
    plt.xlim(xx1.min(),xx1.max())
    plt.ylim(xx2.min(),xx2.max())

    #Plot Class Examples
    for idx,cl in enumerate(np.unique(y)):
        plt.scatter(
            x=X[y==cl,0],
            y=X[y==cl,1],
            alpha=.8,
            c=colors[idx],
            marker=markers[idx],
            label=f'Class {cl}',
            edgecolor='black',
        )

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
lr = LogisticRegression(multi_class='ovr',random_state=1,solver='lbfgs')

X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.fit_transform(X_test_std)

lr.fit(X_train_pca,y_train)

plot_decision_regions(X_train_pca,y_train,classifier=lr)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

In [ ]:
plot_decision_regions(X_test_pca*-1,y_test,classifier=lr)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

In [ ]:
# If we want to see the explained variance ratio of our principal components, we just need to init pca like this
pca2 = PCA(n_components=None)
X_train_pca = pca2.fit_transform(X_train_std)
pca2.explained_variance_ratio_

### Assessing Feature Contributions to each principal component via factor loading

In [ ]:
loadings = eigen_vecs * np.sqrt(eigen_vals)
fig,ax = plt.subplots()
ax.bar(range(13),loadings[:,0],align='center')
ax.set_ylabel('Loadings for PC 1')
ax.set_xticks(range(13))
ax.set_xticklabels(df_wine.columns[1:],rotation=90)
plt.ylim([-1,1])
plt.tight_layout()
plt.show()

# Positive loadings implies a positive correlation and vice versa

In [ ]:
sklearn_loadings = (pca.components_.T * np.sqrt(pca.explained_variance_))*-1
fig,ax = plt.subplots()
ax.bar(range(13),sklearn_loadings[:,0],align='center')
ax.set_ylabel('Loadings for PC 1')
ax.set_xticks(range(13))
ax.set_xticklabels(df_wine.columns[1:],rotation=90)
plt.ylim([-1,1])
plt.tight_layout()
plt.show()

## Supervised Data Compression via Linear Discriminant Analysis

### Computation of Scatter Matricies

#### Compute Mean Vectors for each class label

In [ ]:
np.set_printoptions(precision=4)
mean_vecs = []
for label in range(1,4):
    mean_vecs.append(
        np.mean(
            X_train_std[y_train==label],axis=0
        )
    )
    print(f'MV {label}: {mean_vecs[label-1]}.\n')

#### Compute within class scatter matricies 

In [ ]:
d = 13 #Number of features
S_W = np.zeros((d,d))
for label,mv in zip(range(1,4),mean_vecs):
    class_scatter = np.zeros((d,d))
    for row in X_train_std[y_train == label]:
        row,mv = row.reshape(d,1),mv.reshape(d,1)
        class_scatter += (row-mv).dot((row-mv).T)
    S_W+=class_scatter
print('Within-class Scatter Matrix:',f'{S_W.shape[0]}x{S_W.shape[1]}')

In [ ]:
# One assumption that is made when calculating the scatter matricies is that our class labels are uniformly-distributed, however that assumption is violated
print('Class Label Distribution:',np.bincount(y_train)[1:])

# Thus, we need to scale the individual scatter matricies before summing them up as the scatter matrix
d = 13 
S_W = np.zeros((d,d))
for label,mv in zip(range(1,4),mean_vecs):
    class_scatter = np.cov(
        X_train_std[y_train == label].T
    )
    S_W+= class_scatter
print('Within-class Scatter Matrix:',f'{S_W.shape[0]}x{S_W.shape[1]}')

#### Computer Between-class Scatter Matrix

In [ ]:
mean_overall = np.mean(X_train_std,axis=0)
mean_overall = mean_overall.reshape(d,1)

d = 13
S_B = np.zeros((d,d))
for i, mean_vec in enumerate(mean_vecs):
    n = X_train_std[y_train == i+1,:].shape[0]
    mean_vec = mean_vec.reshape(d,1)
    S_B += (n * (mean_vec - mean_overall).dot((mean_vec - mean_overall).T))
print('Between-class Scatter Matrix:',f'{S_B.shape[0]}x{S_B.shape[1]}')

### Selecting Linear Discriminants for the New Feature Subspace

In [ ]:
eigen_vals,eigen_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
eigen_pairs = [(np.abs(eigen_vals[i]),eigen_vecs[:,i]) for i in range(len(eigen_vals))]
eigen_pairs = sorted(eigen_pairs,key=lambda k: k[0],reverse=True)
print('Eigenvalues in descending order:\n')
for eigen_val in eigen_pairs:
    print(eigen_val[0])

#### Plot of Discriminability for each linear discriminate for each class label

In [ ]:
tot = sum(eigen_vals.real)
discr = [(i/tot) for i in sorted(eigen_vals.real,reverse=True)]
cum_discr = np.cumsum(discr)
plt.bar(range(1,14),discr,align='center',label='Individual Discriminability')
plt.step(range(1,14),cum_discr,where='mid',label='Cumulative Discriminability')
plt.ylabel('Discriminability Ratio')
plt.xlabel('Linear Discriminants')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
#Since the first two discriminants capture 100% of the feature space, we need to select those
w = np.hstack(
    (
        eigen_pairs[0][1][:,np.newaxis].real,
        eigen_pairs[1][1][:,np.newaxis].real
    )
)
print('Matrix W:\n',w)

### Projecting our examples onto the new feature space

In [ ]:
X_train_lda = X_train_std.dot(w)
colors = ['r','b','g']
markers = ['o','s','^']

for l,c,m in zip(np.unique(y_train),colors,markers):
    plt.scatter(
        X_train_lda[y_train==l,0],
        X_train_lda[y_train==l,1]*(-1),
        c=c,label=f'Class {l}',marker=m
    )
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

#Our classes are now perfectly linearly seperable

## Supervised Data Compression via Linear Discriminant Analysis (Implemented using Sklearn)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train_std,y_train)

lr = LogisticRegression(multi_class='ovr',random_state=1,solver='lbfgs')
lr = lr.fit(X_train_lda,y_train)
plot_decision_regions(X_train_lda,y_train,classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

In [ ]:
# Model is 100% accurate on our 2D dataset, instead of our 13D dataset, thanks to LDA
X_test_lda = lda.transform(X_test_std)
plot_decision_regions(X_test_lda,y_test,classifier=lr)
plt.xlabel('LD 1')
plt.ylabel('LD 2')
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

# Non-Linear Dimensionality Reduction

## T-Distributed Stochastic Neighbor Embedding (t-SNE)

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
fig,ax = plt.subplots(1,4)
for i in range(4):
    ax[i].imshow(digits.images[i],cmap='Greys')
plt.show()

In [ ]:
y_digits = digits.target
X_digits = digits.data

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2,init='pca',random_state=123)
X_digits_tsne = tsne.fit_transform(X_digits)

In [ ]:
import matplotlib.patheffects as PathEffects
def plot_projection(x,colors):
    f = plt.figure(figsize=(8,8))
    ax = plt.subplot(aspect='equal')
    for i in range(10):
        plt.scatter(
            x[colors==i,0],
            x[colors==i,1]
        )
    
    for i in range(10):
        xtext,ytext = np.median(x[colors==i,:],axis=0)
        txt = ax.text(xtext,ytext,str(i),fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5,foreground='w'),
            PathEffects.Normal()
        ])
plot_projection(X_digits_tsne,y_digits)
plt.show()